## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.6666,-52.7314,20.57,82,40,11.50,scattered clouds
1,1,Puerto Colombia,CO,10.9878,-74.9547,80.69,81,24,20.47,few clouds
2,2,Maridi,SS,4.9158,29.4742,68.70,22,96,4.18,overcast clouds
3,3,Burgeo,CA,47.6167,-57.6152,29.48,70,100,18.37,overcast clouds
4,4,Uni,RU,57.7535,51.4923,14.32,97,100,2.55,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Colombia,CO,10.9878,-74.9547,80.69,81,24,20.47,few clouds
5,5,Catamarca,AR,-28.4696,-65.7852,80.78,45,15,5.99,few clouds
28,28,Lorengau,PG,-2.0226,147.2712,83.95,76,95,7.99,overcast clouds
40,40,Butaritari,KI,3.0707,172.7902,82.24,74,62,16.96,broken clouds
42,42,Tautira,PF,-17.7333,-149.1500,86.02,79,100,20.67,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
new_preferred_cities_df.count()

City_ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Colombia,CO,80.69,few clouds,10.9878,-74.9547,
5,Catamarca,AR,80.78,few clouds,-28.4696,-65.7852,
28,Lorengau,PG,83.95,overcast clouds,-2.0226,147.2712,
40,Butaritari,KI,82.24,broken clouds,3.0707,172.7902,
42,Tautira,PF,86.02,light rain,-17.7333,-149.1500,
68,Poum,NC,81.25,overcast clouds,-20.2333,164.0167,
71,Isangel,VU,86.11,overcast clouds,-19.5500,169.2667,
83,Kahului,US,82.42,few clouds,20.8947,-156.4700,
93,Unguia,CO,82.42,broken clouds,8.0434,-77.0945,
108,Vao,NC,80.62,broken clouds,-22.6667,167.4833,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Colombia,CO,80.69,few clouds,10.9878,-74.9547,Hotel El Emigrante
5,Catamarca,AR,80.78,few clouds,-28.4696,-65.7852,Amerian Catamarca Park Hotel
28,Lorengau,PG,83.95,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
40,Butaritari,KI,82.24,broken clouds,3.0707,172.7902,Isles Sunset Lodge
42,Tautira,PF,86.02,light rain,-17.7333,-149.1500,Pension Oaoa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))